This notebooks aims to creates a boosted dataset of fake conversations between a user and an assistant about a given list of acronym and their definitions.

If you want to skip this part, you can used pre-cooked training dataset located in [../example_data/train_dataset.json](../example_data/train_dataset.json). Same for test dataset : [../example_data/test_dataset.json](../example_data/test_dataset.json).

You need to have access to a ollama server in order to run this notebook. See README.md to install one.

## 0 - Loads config

In [ ]:
import os

ollama_url = "http://localhost:11434"
data_dir = "./example_data"
model_name = "gemma3:4b"

if not os.path.isfile(os.path.join(data_dir, "acronym.json")):
    raise FileNotFoundError(f"Please add a json named acronym.json in dir {data_dir}. If you don't have one, you can copy one from example_data dir.")

print(
    f"""
    ollama_url: {ollama_url},
    LLM used for data generation : {model_name},
    loading data from : {data_dir},
"""
)

## 1 - Request model with ollama

> The ollama server should be started in order to run those cells. Run `ollama serve` in a terminal window if not already done.

In [ ]:
from ollama import generate

generate(model=model_name, prompt="How much is 1+1 ?").response

__Exercise :__

- Ask the model anything

## 2 - Creates custom prompt and asks a LLM to boost our dataset

In [ ]:
def create_acronym_prompt(n_conv, acro, definition):
    """
    Returns prompt to get synthethic conversation about acronym
    and definitions.
    """
    return (
        f"Create {n_conv} fictive conversations between an user and an assistant.\n"
        "Those conversations must contains 1 question and 1 answer.\n"
        f"Each question must be an user asking for the definition the term {acro}; and each answer must contain the definition : '{definition}'.\n"
        "All the conversations must be somehow diverse.\n"
        "I want only questions that ask the definition, not more. \n"
        "Each conversation will be formatted in a json list, of the form :"
        "[\n"
        "  {\n"
        "     'role': 'user'',\n"
        "     'content': THE QUESTION\n"
        "  },\n"
        "  {\n"
        "     'role': 'assistant',\n"
        "     'content': THE ANSWER\n"
        "  }\n"
        "] \n"
        "Hence, the final result will look like : \n"
        "[\n"
        "   [\n"
        "       {\n"
        "           'role': 'user'',\n"
        "           'content': THE FIRST QUESTION\n"
        "       },\n"
        "       {\n"
        "           'role': 'assistant',\n"
        "           'content': THE ANSWER TO THE FIRST QUESTION\n"
        "       }\n"
        "   ], \n"
        "   [\n"
        "       {\n"
        "           'role': 'user'',\n"
        "           'content': THE SECOND QUESTION\n"
        "       },\n"
        "       {\n"
        "           'role': 'assistant',\n"
        "           'content': THE ANSWER TO THE SECOND QUESTION\n"
        "       }\n"
        "   ], \n"
        "   [\n"
        "       {\n"
        "           'role': 'user'',\n"
        "           'content': THE LAST QUESTION\n"
        "       },\n"
        "       {\n"
        "           'role': 'assistant',\n"
        "           'content': THE ANSWER TO THE LAST QUESTION\n"
        "       }\n"
        "   ] \n"
        "]\n"
        f"Keep it short. I remind you that you must give {n_conv} conversations.\n"
        "Your final answer must be only the raw json; no fioritures.\n"
    )

prompt = create_acronym_prompt(
    2, acro="HPS", definition="Herbs, Pasta, Spices: A cooking approach that heavily incorporates herbs and spices with pasta dishes."
)
print(prompt)

In [ ]:
answer = generate(model=model_name, prompt=prompt).response
print(answer)


The output of the LLM could be anything, but we would rather want to check the format, to avoid future errors.
We can give ollama.generate function a scheme, which will structure the output of the LLM.

In [ ]:
from ollama import generate
import json

# We define a scheme for our list of Conversations, for type-checking.
# The syntax for writing scheme is presented raw here, but usually
# it is extracted from pydantic BaseModel subclasses.

scheme_llm_output = {
    "$defs": {
        "ConversationModel": {
            "title": "ConversationModel",
            "items": {
                "$ref": "#/$defs/MessageModel",
            },
            "title": "Messages",
            "type": "array",
        },
        "MessageModel": {
            "properties": {
                "role": {
                    "enum": ["user", "assistant"],
                    "title": "Role",
                    "type": "string",
                },
                "content": {"title": "Content", "type": "string"},
            },
            "required": ["role", "content"],
            "title": "MessageModel",
            "type": "object",
        },
    },
    "items": {"$ref": "#/$defs/ConversationModel"},
    "title": "ConversationListModel",
    "type": "array"
}

structured_output = json.loads(generate(model=model_name, prompt=prompt, format=scheme_llm_output).response)

print(structured_output)
print(len(structured_output))

__Exercises :__

- change the prompt to get : 

    - more verbose answers, or more concise ones,

    - answers in an other language,

    - answers that specify the field of the acronym (e.g. cooking, science, ...) 



## Create training and test dataset

We load our structured list of acronyms and their definitions, and we create a datasets of conversations for the training and test of the LLM.

In [ ]:
import json
import random
import os

# loads list of acronyms and their definitions
raw_data_dir = os.path.join(data_dir, "acronym.json")

with open(raw_data_dir, "rt") as f:
    raw_data = json.load(f)
    
n_acros = len(raw_data)
print(f"Example of dataset element : \n {json.dumps(raw_data[random.randint(0, len(raw_data)-1)], indent=4)}")

In [ ]:
from tqdm import tqdm # for progress bars

train_dataset = []
test_dataset = []
n_convs_per_acro = 4

for each_elem in tqdm(raw_data):
    acronym = each_elem["acronym"]
    definition = each_elem["definition"]
    prompt = create_acronym_prompt(n_convs_per_acro, acronym, definition)
    structured_output = json.loads(generate(model=model_name, prompt=prompt, format=scheme_llm_output).response)
    if len(structured_output) < n_convs_per_acro:
        print(f"Skipping acronym {acronym}")
        continue

    train_conv = structured_output[:n_convs_per_acro-1]
    eval_conv = [structured_output[n_convs_per_acro-1]]

    train_elem = {
        "acronym": acronym,
        "ground_truth": definition,
        "conversation": train_conv,
    }
    test_elem = {
        "acronym": acronym,
        "ground_truth": definition,
        "conversation": eval_conv,
    }

    train_dataset.append(train_elem)
    test_dataset.append(test_elem)

In [ ]:
train_dataset[8] # example

Then we save the dataset.

In [ ]:
train_data_dir = "../bucket/fine_tuning_acronym/data/train_dataset.json"
test_data_dir = "../bucket/fine_tuning_acronym/data/test_dataset.json"

# saves into a single json all training conversations
with open(train_data_dir, "wt") as f:
    json.dump(train_dataset, f, indent=4) # avoid indent param for bigger datasets

# saves into a single json all test conversations
with open(test_data_dir, "wt") as f:
    json.dump(test_dataset, f, indent=4) # avoid indent param for bigger datasets